In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [2]:
cd ../

D:\git_repos\lab_data_science


In [5]:
#TRAIN - VAL

In [3]:
import src.config as cfg
from src.utils import save_as_pickle, load_as_pickle

In [10]:
# raw_path = 'data/raw/'
# train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
# test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [11]:
# TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
# ID_COL = 'ID'
# EDU_COL = 'Образование'
# SEX_COL = 'Пол'
# CAT_COLS = [
#     'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
#     'Профессия', 'Статус Курения', 'Алкоголь',
#     'Время засыпания', 'Время пробуждения'
# ]
# OHE_COLS = [
#     'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
#     'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
#     'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
#     'Спорт, клубы', 'Религия, клубы'
# ]
# REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког', 'Частота пасс кур']

In [12]:
# def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
#     df = df.set_index(idx_col)
#     return df

In [13]:
# train = train.set_index(ID_COL)
# test = test.set_index(ID_COL)

In [14]:
# train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]

In [15]:
# train.pipe()

In [16]:
# def preprocess(df: pd.DataFrame) -> pd.DataFrame:
#     df = set_idx(df, ID_COL)
#     df = drop_unnecesary_id(df)
#     df = fill_sex(df)
#     df = cast_types(df)
#     return df
    
    

In [17]:
# def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
#     if 'ID_y' in df.columns:
#         df = df.drop('ID_y', axis=1)
#     return df

In [18]:
# train = drop_unnecesary_id(train)

In [19]:
# def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
#     df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
#     return df

In [20]:
# train = add_ord_edu(train)
# test = add_ord_edu(test)

In [21]:
# def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
#     most_freq = df[SEX_COL].value_counts().index[0]
#     df[SEX_COL] = df[SEX_COL].fillna(most_freq)
#     return df

In [22]:
# def cast_types(df: pd.DataFrame) -> pd.DataFrame:
#     df[CAT_COLS] = df[CAT_COLS].astype('category')

#     ohe_int_cols = train[OHE_COLS].select_dtypes('number').columns
#     df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

#     df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
#     return df
    

In [4]:
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

In [24]:
# train_data_path = 'data/interim/train.pkl'
# target_data_path = 'data/interim/target.pkl'

In [25]:
# def load_splitted_data(train_path, target_path):
#     train = load_as_pickle(train_path)
#     target = load_as_pickle(target_path)
#     return train, target

In [26]:
# train_data, target_data = load_splitted_data(train_data_path, target_data_path)

In [27]:
# x_train, x_test, y_train, y_test = train_test_split(train_data, target_data, test_size=0.25)

In [28]:
# path_to_splitted_train_data = 'data/processed/train.pkl'
# path_to_splitted_train_data_target = 'data/processed/train_target.pkl'
# path_to_splitted_val_data = 'data/processed/val.pkl'
# path_to_splitted_val_data_target = 'data/processed/val_target.pkl'

In [29]:
# save_as_pickle(x_train, path_to_splitted_train_data)
# save_as_pickle(y_train, path_to_splitted_train_data_target)
# save_as_pickle(x_test, path_to_splitted_val_data)
# save_as_pickle(y_test, path_to_splitted_val_data_target)

In [30]:
#FEATURE GENERATION

In [5]:
from src.config import (path_to_splitted_train_data,
                path_to_splitted_train_data_target,
                path_to_splitted_val_data,
                path_to_splitted_val_data_target)

In [6]:
x_train = load_as_pickle(path_to_splitted_train_data)
y_train = load_as_pickle(path_to_splitted_train_data_target)
x_val = load_as_pickle(path_to_splitted_val_data)
y_val = load_as_pickle(path_to_splitted_val_data_target)

In [13]:
import src.features.utils as feat_utils
from sklearn.pipeline import Pipeline
from src.utils import load_model_as_pickle

In [15]:
RFC = load_model_as_pickle('models/' + cfg.name_lr_model)

In [20]:
RFC_pipeline = Pipeline([('column_filler', feat_utils.empty_column_filler()),
                            ('target_encoder', feat_utils.TargetEncoder(cfg.CAT_COLS, 5)),
                            ('model', RFC)])

C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [22]:
RFC_pipeline.fit(x_train, y_train)

Pipeline(steps=[('column_filler',
                 <src.features.utils.empty_column_filler object at 0x0000023BC61A8940>),
                ('target_encoder',
                 <src.features.utils.TargetEncoder object at 0x0000023BC61A88E0>),
                ('model',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100)))])

In [23]:
RFC_pipeline.predict(x_val)

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int8)

In [33]:
import category_encoders as ce

In [25]:
pipline = load_model_as_pickle('models/' + cfg.name_RFC_pipeline)
pipline.predict(x_val)

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int8)

In [34]:
def second_hand_smoke_count(value):
    if value == '1-2 раза в неделю':
        return 1.5
    if value == '2-3 раза в день':
        return 17.5
    if value == '3-6 раз в неделю':
        return 4.5
    if value == '4 и более раз в день':
        return 28.0
    if value == 'не менее 1 раза в день':
        return 7.0
    return 0.0

In [35]:
def fill_real_cols_with_zero(df):
    df = df.fillna(0.)
    return df

def fill_real_cols(data):
    data.loc[:, cfg.REAL_COLS[:-1]] = fill_real_cols_with_zero(data.loc[:, cfg.REAL_COLS[:-1]])
    data['Частота пасс кур'] = [second_hand_smoke_count(i) for i in data['Частота пасс кур']]
    return data

In [36]:
x_train = fill_real_cols(x_train)

In [37]:
class empty_column_filler:
    def second_hand_smoke_count(self, value):
        if value == '1-2 раза в неделю':
            return 1.5
        if value == '2-3 раза в день':
            return 17.5
        if value == '3-6 раз в неделю':
            return 4.5
        if value == '4 и более раз в день':
            return 28.0
        if value == 'не менее 1 раза в день':
            return 7.0
        return 0.0
    
    
    def fill_real_cols_with_zero(self, df):
        df = df.fillna(0.)
        return df

    def fill_real_cols(self, data):
        data.loc[:, cfg.REAL_COLS[:-1]] = fill_real_cols_with_zero(data.loc[:, cfg.REAL_COLS[:-1]])
        data['Частота пасс кур'] = [second_hand_smoke_count(i) for i in data['Частота пасс кур']]
        return data


    def fit(self, x, y=None):
        x = self.fill_real_cols(x)

    def transform(self, x, y=None):
        x = self.fill_real_cols(x)
        return x

    def fit_transform(self, x, y=None):
        return self.transform(x)

In [38]:
# def target_encode_multiclass(X,y): #X,y are pandas df and series
#     y=y.astype(str)  #convert to string to onehot encode
#     enc=ce.OneHotEncoder().fit(y)
#     y_onehot=enc.transform(y) 

#     class_names=y_onehot.columns  #names of onehot encoded columns

#     X_obj=X.select_dtypes('object') #separate categorical columns
#     X=X.select_dtypes(exclude='object')

#     for class_ in class_names:

#         enc=ce.TargetEncoder()
#         enc.fit(X_obj,y_onehot[class_]) #convert all categorical
#         temp=enc.transform(X_obj)       #columns for class_
#         temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
#         X=pd.concat([X,temp],axis=1)    #add to original dataset

#     return X

In [39]:
# class TargetEncoder(BaseEstimator, TransformerMixin):
class TargetEncoder():
    
    def __init__(self, cat_cols, num_classes=1):
        self.te = [ce.TargetEncoder(drop_invariant=False)] * num_classes
        self.cat_cols = cat_cols
        
        
    def fit(self, x, y):
        for target_num in range(num_classes):
            target_col = cfg.TARGET_COLS[target_num]
            self.te[target_num].fit(x[self.cat_cols], y_train[target_col])

        
    def transform(self, x, y=None):
        target_encoded_x = x.copy()
        for target_num in range(num_classes):
            target_col = cfg.TARGET_COLS[target_num]
            encoded_cols = self.te[target_num].transform(x[self.cat_cols])
            
            target_encoded_x[[col + "/" + target_col for col in self.cat_cols]] = encoded_cols

        target_encoded_x = target_encoded_x.drop(self.cat_cols, axis=1)
        return target_encoded_x
            
    def fit_transform(self, x, y=None):
        self.fit(x, y)
        return self.transform(x, y)

In [40]:
target_encoder = TargetEncoder(cfg.CAT_COLS, 5)

C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [41]:
# def target_encoding(cat_features, x_train, y_train=None, test=False):
#     target_encoded_x_train = x_train.copy()
#     for target_num in range(num_classes):
#         target_col = cfg.TARGET_COLS[target_num]
#         if test:
#             encoded_cols = te[target_num].transform(x_train[cat_features])
#         else:
#             encoded_cols = te[target_num].fit_transform(x_train[cat_features], y_train[target_col])
            
#         target_encoded_x_train[[col + "/" + target_col for col in cfg.CAT_COLS]] = encoded_cols

#     target_encoded_x_train = target_encoded_x_train.drop(cat_features, axis=1)
#     return target_encoded_x_train

In [42]:
num_classes = 5

In [43]:

#     target_encoded_x_train.insert(loc=target_encoded_x_train.columns[:-1],
#                                   column=[col + "/" + target_col for col in cfg.CAT_COLS],
#                                   value=encoded_cols,
#                                   allow_duplicates = False)
        

In [44]:
x_val.info

<bound method DataFrame.info of               Пол                                      Семья        Этнос  \
ID                                                                          
54-101-042-01   Ж                  в браке в настоящее время  европейская   
54-102-020-01   М                  в браке в настоящее время  европейская   
54-102-128-02   Ж                  в браке в настоящее время  европейская   
54-002-027-01   Ж                  в браке в настоящее время  европейская   
54-102-344-01   Ж                  в браке в настоящее время  европейская   
...            ..                                        ...          ...   
54-701-041-01   Ж                  в браке в настоящее время  европейская   
54-001-017-01   Ж                             вдовец / вдова  европейская   
54-002-050-01   Ж                                  в разводе  европейская   
54-102-263-01   Ж  гражданский брак / проживание с партнером  европейская   
54-504-029-01   М                  в браке в

In [45]:
from sklearn.pipeline import Pipeline

In [47]:
# lr_pipeline = Pipeline([('column_filler', empty_column_filler()),
#                      ('target_encoder', TargetEncoder(cfg.CAT_COLS, 5)),
#                      ('model', MultiOutputClassifier(LogisticRegression(max_iter=10000)))])
# lr_pipeline.fit(x_train, y_train)
# # pipeline.predict(x_train)

In [ ]:
# lr_pipeline.predict(x_val)

In [ ]:
# target_encoded_x_train = target_encoder.fit_transform(x_train, y_train)

# target_encoded_x_test = target_encoder.transform(x_test, y_test)
# target_encoded_x_test

In [ ]:
#MODELS

In [30]:
import catboost

In [32]:
def get_indexes_of_cat_columns(df: pd.DataFrame, col_names: str):
    idx = []
    for col in col_names:
        idx.append(df.columns.tolist().index(col))
        
    return idx

In [33]:
cat_idx = get_indexes_of_cat_columns(x_train, cfg.CAT_COLS)
cat_idx

[0, 1, 2, 3, 4, 5, 6, 20, 25, 27, 28]

In [84]:
from sklearn.model_selection import GridSearchCV

In [98]:
cat_idx

[0, 1, 2, 3, 4, 5, 6, 20, 25, 27, 28]

In [152]:
param_grid = {
    'model_size_reg': [0, 1],
    'n_estimators' : [100, 500, 1000],
    'one_hot_max_size' : [0, 1000, 2000],
    'max_ctr_complexity': [1, 6]
}

grid = GridSearchCV(catboost.CatBoostClassifier(cat_features=cat_idx, loss_function='MultiLogloss', verbose=100),
                    param_grid, cv=3, scoring='recall', 
                    verbose=0)

grid.fit(x_train, y_train)
grid.predict(x_val)

C:\git\lab_data_science\venv\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Learning rate set to 0.062039
0:	learn: 0.6417310	total: 47.8ms	remaining: 4.73s
99:	learn: 0.2120689	total: 4.41s	remaining: 0us
Learning rate set to 0.062039
0:	learn: 0.6477840	total: 24ms	remaining: 2.37s


C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1920, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1563, in precision_

99:	learn: 0.2210599	total: 4.65s	remaining: 0us
Learning rate set to 0.062094
0:	learn: 0.6471926	total: 13.9ms	remaining: 1.37s


C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1920, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1563, in precision_

...

C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1920, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1563, in precision_

99:	learn: 0.2216165	total: 8.85s	remaining: 0us
Learning rate set to 0.073787


C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1920, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\git\lab_data_science\venv\lib\site-packages\sklearn\metrics\_classification.py", line 1563, in precision_

0:	learn: 0.6363880	total: 48.2ms	remaining: 4.77s
99:	learn: 0.2172801	total: 5.68s	remaining: 0us


array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [173]:
grid.score(x_val, y_val)

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

In [35]:
a = catboost.CatBoostClassifier(cat_features=cat_idx, loss_function='MultiLogloss', verbose=100)

In [36]:
a.fit(x_train, y_train)

C:\git\lab_data_science\venv\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Learning rate set to 0.008933
0:	learn: 0.6860496	total: 390ms	remaining: 6m 29s
100:	learn: 0.3834254	total: 5.46s	remaining: 48.6s
200:	learn: 0.3291134	total: 12.5s	remaining: 49.8s
300:	learn: 0.3055849	total: 21.2s	remaining: 49.1s
400:	learn: 0.2863347	total: 31.4s	remaining: 47s
500:	learn: 0.2702892	total: 42.3s	remaining: 42.1s
600:	learn: 0.2556305	total: 53s	remaining: 35.2s
700:	learn: 0.2403039	total: 1m 4s	remaining: 27.5s
800:	learn: 0.2218298	total: 1m 15s	remaining: 18.8s
900:	learn: 0.2054136	total: 1m 26s	remaining: 9.53s
999:	learn: 0.1911284	total: 1m 37s	remaining: 0us


In [37]:
a.feature_importances_

array([3.53480982e-01, 8.58741512e+00, 3.01550424e+00, 2.54710485e+00,
       4.88558543e+00, 8.54372056e+00, 1.38509950e+01, 2.84069226e+00,
       1.89945142e+00, 2.35593804e-01, 1.14547457e+00, 9.54152469e-02,
       4.66327021e-02, 5.10910311e-02, 4.34226486e-02, 1.29522786e-02,
       2.26589090e-02, 1.34098481e+01, 3.78059987e-02, 1.20183223e+00,
       8.27387620e+00, 4.71981423e-01, 5.39887147e-01, 5.07155891e-02,
       4.70918331e-01, 8.00022277e+00, 9.10733584e-01, 8.36543777e+00,
       9.60799485e+00, 4.19440494e-01, 2.51346896e-02, 3.69796929e-02])

In [41]:
rfC = MultiOutputClassifier(RandomForestClassifier(max_depth = 1000))
rfC.fit(x_train, y_train)

ValueError: could not convert string to float: 'Ж'

In [144]:
max_depth_values = [100, 500, 1000]

best_score = 0
best_model = None

for i in max_depth_values:
    rfC = MultiOutputClassifier(RandomForestClassifier(max_depth = i))

    rfC.fit(te.fit_transform(x_train, y_train), y_train)
    y_pred = rfC.predict(te.transform(fill.transform(x_val)))
    score = recall_score(y_val, preds, average='micro')
    if score > best_score:
        best_score = score
        best_model = rfC
    

In [145]:
best_score

0.37681159420289856

In [43]:
from src.features.utils import TargetEncoder, empty_column_filler

In [44]:
a = MultiOutputClassifier(RandomForestClassifier())

te = TargetEncoder(cfg.CAT_COLS, 5)
fill = empty_column_filler()
x_train = fill.fit_transform(x_train)

a.fit(te.fit_transform(x_train, y_train), y_train)

C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


MultiOutputClassifier(estimator=RandomForestClassifier())

In [49]:
a.fit(te.fit_transform(x_train, y_train), y_train)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [45]:
preds = a.predict(te.transform(fill.transform(x_val)))

In [52]:
a.estimator.fit(te.fit_transform(x_train, y_train), y_train)
a.estimator.feature_importances_

array([0.01853008, 0.02157847, 0.00693141, 0.01709108, 0.00718598,
       0.00578241, 0.00678756, 0.00628692, 0.00303272, 0.00020596,
       0.06057083, 0.00498008, 0.01595014, 0.02325055, 0.01904295,
       0.01293618, 0.        , 0.04661357, 0.01212465, 0.00834934,
       0.00233907, 0.00495155, 0.01422034, 0.0006821 , 0.00469978,
       0.00634493, 0.01210676, 0.02442504, 0.00843774, 0.00956835,
       0.02620389, 0.02665562, 0.00639806, 0.01348091, 0.0008475 ,
       0.0047669 , 0.00597276, 0.01134444, 0.02483728, 0.00762436,
       0.01093402, 0.02684346, 0.02568942, 0.00498617, 0.01434467,
       0.00079481, 0.00512016, 0.00534969, 0.01134583, 0.02515356,
       0.00966359, 0.00874217, 0.02766255, 0.02717771, 0.00553591,
       0.01373813, 0.00125981, 0.00451615, 0.00599432, 0.01193236,
       0.02533779, 0.00939681, 0.00977809, 0.02805299, 0.02671783,
       0.00681968, 0.0138934 , 0.0009449 , 0.00489978, 0.00541521,
       0.01254731, 0.02460614, 0.00823285, 0.00988025, 0.02602

In [128]:
count_metrics(y_val, preds)

0.6554621848739496
0.37681159420289856
0.4785276073619632
0.6676568092471982


In [94]:
for i in range(100, 500, 100):
    boosting_model = catboost.CatBoostClassifier(
        loss_function='MultiLogloss',
        cat_features=cat_idx,
        model_size_reg=0,
        n_estimators = i
    )
    boosting_model.fit(x_train, y_train)
    

Learning rate set to 0.073787
0:	learn: 0.6352459	total: 397ms	remaining: 39.3s
1:	learn: 0.5882564	total: 460ms	remaining: 22.5s
2:	learn: 0.5508143	total: 480ms	remaining: 15.5s
3:	learn: 0.5186669	total: 502ms	remaining: 12s
4:	learn: 0.4935667	total: 549ms	remaining: 10.4s
5:	learn: 0.4632556	total: 606ms	remaining: 9.49s
6:	learn: 0.4461408	total: 632ms	remaining: 8.39s
7:	learn: 0.4323914	total: 644ms	remaining: 7.41s
8:	learn: 0.4207278	total: 664ms	remaining: 6.71s
9:	learn: 0.4046415	total: 695ms	remaining: 6.26s
10:	learn: 0.3969424	total: 717ms	remaining: 5.8s
11:	learn: 0.3855813	total: 733ms	remaining: 5.38s
12:	learn: 0.3745002	total: 789ms	remaining: 5.28s
13:	learn: 0.3677874	total: 842ms	remaining: 5.17s
14:	learn: 0.3632742	total: 902ms	remaining: 5.11s
15:	learn: 0.3543569	total: 954ms	remaining: 5.01s
16:	learn: 0.3492747	total: 994ms	remaining: 4.85s
17:	learn: 0.3472954	total: 1.01s	remaining: 4.61s
18:	learn: 0.3427385	total: 1.06s	remaining: 4.54s
19:	learn: 0.3

61:	learn: 0.3054811	total: 4.83s	remaining: 10.8s
62:	learn: 0.3045518	total: 4.95s	remaining: 10.8s
63:	learn: 0.3037400	total: 5.06s	remaining: 10.7s
64:	learn: 0.3027030	total: 5.16s	remaining: 10.7s
65:	learn: 0.3016972	total: 5.26s	remaining: 10.7s
66:	learn: 0.3010255	total: 5.32s	remaining: 10.6s
67:	learn: 0.2995617	total: 5.42s	remaining: 10.5s
68:	learn: 0.2986010	total: 5.53s	remaining: 10.5s
69:	learn: 0.2972961	total: 5.65s	remaining: 10.5s
70:	learn: 0.2956588	total: 5.79s	remaining: 10.5s
71:	learn: 0.2946536	total: 5.9s	remaining: 10.5s
72:	learn: 0.2936862	total: 6.03s	remaining: 10.5s
73:	learn: 0.2922703	total: 6.2s	remaining: 10.6s
74:	learn: 0.2912202	total: 6.34s	remaining: 10.6s
75:	learn: 0.2904992	total: 6.48s	remaining: 10.6s
76:	learn: 0.2897221	total: 6.63s	remaining: 10.6s
77:	learn: 0.2885939	total: 6.78s	remaining: 10.6s
78:	learn: 0.2873492	total: 6.91s	remaining: 10.6s
79:	learn: 0.2857658	total: 7.03s	remaining: 10.6s
80:	learn: 0.2847329	total: 7.17s

24:	learn: 0.4137033	total: 1.57s	remaining: 17.3s
25:	learn: 0.4097519	total: 1.68s	remaining: 17.7s
26:	learn: 0.4054525	total: 1.84s	remaining: 18.6s
27:	learn: 0.4019708	total: 1.91s	remaining: 18.5s
28:	learn: 0.3986980	total: 1.93s	remaining: 18.1s
29:	learn: 0.3933304	total: 2.06s	remaining: 18.5s
30:	learn: 0.3908412	total: 2.08s	remaining: 18.1s
31:	learn: 0.3872544	total: 2.14s	remaining: 17.9s
32:	learn: 0.3834125	total: 2.16s	remaining: 17.5s
33:	learn: 0.3792367	total: 2.25s	remaining: 17.6s
34:	learn: 0.3773804	total: 2.28s	remaining: 17.3s
35:	learn: 0.3751354	total: 2.37s	remaining: 17.4s
36:	learn: 0.3734991	total: 2.39s	remaining: 17s
37:	learn: 0.3707336	total: 2.51s	remaining: 17.3s
38:	learn: 0.3679052	total: 2.57s	remaining: 17.2s
39:	learn: 0.3646367	total: 2.72s	remaining: 17.7s
40:	learn: 0.3618058	total: 2.89s	remaining: 18.3s
41:	learn: 0.3594378	total: 3.06s	remaining: 18.8s
42:	learn: 0.3575622	total: 3.21s	remaining: 19.2s
43:	learn: 0.3552448	total: 3.3s	

186:	learn: 0.2553568	total: 19.7s	remaining: 11.9s
187:	learn: 0.2551196	total: 19.9s	remaining: 11.8s
188:	learn: 0.2550395	total: 20.1s	remaining: 11.8s
189:	learn: 0.2547528	total: 20.3s	remaining: 11.7s
190:	learn: 0.2547441	total: 20.3s	remaining: 11.6s
191:	learn: 0.2543618	total: 20.5s	remaining: 11.5s
192:	learn: 0.2536573	total: 20.7s	remaining: 11.5s
193:	learn: 0.2530790	total: 20.9s	remaining: 11.4s
194:	learn: 0.2523821	total: 21s	remaining: 11.3s
195:	learn: 0.2515422	total: 21.2s	remaining: 11.3s
196:	learn: 0.2508466	total: 21.4s	remaining: 11.2s
197:	learn: 0.2502425	total: 21.5s	remaining: 11.1s
198:	learn: 0.2497470	total: 21.7s	remaining: 11s
199:	learn: 0.2490990	total: 21.8s	remaining: 10.9s
200:	learn: 0.2485704	total: 21.9s	remaining: 10.8s
201:	learn: 0.2481854	total: 22s	remaining: 10.7s
202:	learn: 0.2478119	total: 22.2s	remaining: 10.6s
203:	learn: 0.2471754	total: 22.3s	remaining: 10.5s
204:	learn: 0.2467863	total: 22.4s	remaining: 10.4s
205:	learn: 0.2467

KeyboardInterrupt: 

In [64]:
from sklearn.model_selection import KFold
from catboost import Pool

In [ ]:
#Так как крайне важно правильно предсказать наличие смертельной болезни, для нашей задачи хорошо подойдет метрика recall, показывающая долю верно предсказанных болезней. Также, чтобы выявить количество ложно положительных результатов(то есть случаев, когда модель выявила болезнь у здорового человека), можно использовать метрику presicion. В ккачестве метрики, объединяющей две предыдущие, возьмем roc_auc. 

In [ ]:
catboost_pipeline = Pipeline([('column_filler', empty_column_filler()),
                              ('model', catboost.CatBoostClassifier(
                                            loss_function='MultiLogloss',
                                            cat_features=cat_idx,
                                            verbose=25
                                            )
                              )
                             ])

catboost_pipeline.fit(x_train, y_train)

In [ ]:
preds = catboost_pipeline.predict(x_val)

In [126]:
def count_metrics(y_test, preds):
    print(precision_score(y_test, preds, average='micro'))
    print(recall_score(y_test, preds, average='micro'))
    print(f1_score(y_test, preds, average='micro'))
    print(roc_auc_score(y_test, preds, average='micro'))

In [ ]:
count_metrics(y_val, preds)

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
fCl_target_encode = MultiOutputClassifier(LogisticRegression()).fit(target_encoded_x_train, y_train)
# fCl = MultiOutputClassifier(RandomForestClassifier()).fit(x_train, y_train)

In [ ]:
target_encoded_test = target_encoding(cfg.CAT_COLS, x_test, test=True)
target_encoded_test = fill_real_cols(target_encoded_test)
preds = fCl_target_encode.predict(target_encoded_test)

In [ ]:
count_metrics(y_test, preds)